## Importation et chargement des données avec Pandas

In [46]:
import pandas as pd
df = pd.read_csv(r'../data/clean_data.csv')

df.head()

,text_clean,airline_sentiment,word_count
0,virginamerica what dhepburn said,neutral,4
1,virginamerica plus you ve added commercials t...,positive,9
2,virginamerica i didn t today must mean i n...,neutral,12
3,virginamerica it s really aggressive to blast...,negative,17
4,virginamerica and it s a really big bad thing...,negative,10


## Séparation des données (Train / Test)

In [47]:
from sklearn.model_selection import train_test_split
X = df["text_clean"]
y = df["airline_sentiment"]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

## Vérification de la version de Sentence Transformers installée

In [48]:
import sentence_transformers 
print(sentence_transformers.__version__)

3.0.0
